In [ ]:
pip install scikit-surprise


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import pandas as pd

In [14]:
df_final = pd.read_csv("../data/goodbooks-10k/final_svd_dataset.csv")

In [ ]:
df_final.groupby('decade')['book_id'].count(),uniqeu


decade
1920               77298
1930              121695
1940              133772
1950              254674
1960              249719
1970              244194
1980              443368
1990              847948
2000             1808808
2010             1268418
Ancient Books     520643
Name: book_id, dtype: int64

In [23]:
df_final.groupby('decade')['book_id'].nunique()

decade
1920               89
1930              121
1940              155
1950              210
1960              272
1970              400
1980              704
1990             1360
2000             3121
2010             3067
Ancient Books     480
Name: book_id, dtype: int64

In [ ]:
df_final['rating'].describe()

count    5.970537e+06
mean     3.919667e+00
std      9.910473e-01
min      1.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      5.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [72]:
df_final.columns

Index(['user_id', 'book_id', 'rating', 'decade'], dtype='object')

In [24]:
df_final

,user_id,book_id,rating,decade
0,1,258,5,2000
1,2,4081,4,2000
2,2,260,5,1930
3,2,9296,5,1970
4,2,2318,3,1990
...,...,...,...,...
5970532,49925,510,5,1990
5970533,49925,528,4,1990
5970534,49925,722,4,1990
5970535,49925,949,5,1990


In [ ]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import csv
import os

# ---------------------
# Load dataset
# ---------------------
df_final = pd.read_csv("df_original.csv")  # Ensure the file is in the same directory

# Optional: Convert user_id and book_id to strings (Surprise handles strings better)
df_final['user_id'] = df_final['user_id'].astype(str)
df_final['book_id'] = df_final['book_id'].astype(str)

# ---------------------
# Prepare Surprise Dataset
# ---------------------
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_final[['user_id', 'book_id', 'rating']], reader)

# Split into train and test sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# ---------------------
# Train SVD model
# ---------------------
svd = SVD(n_factors=100, random_state=42)
svd.fit(trainset)

# ---------------------
# Evaluate RMSE
# ---------------------
predictions = svd.test(testset)
rmse = accuracy.rmse(predictions)
# ---------------------
# Generate Top-25 Recommendations per User
# ---------------------
all_user_ids = df_final['user_id'].unique()
all_book_ids = df_final['book_id'].unique()

user_recommendations = {}

print("Generating recommendations...")
for i, user_id in enumerate(all_user_ids):
    rated_books = df_final[df_final['user_id'] == user_id]['book_id'].tolist()
    unseen_books = [book for book in all_book_ids if book not in rated_books]
    
    preds = [(book, svd.predict(user_id, book).est) for book in unseen_books]
    top_preds = sorted(preds, key=lambda x: x[1], reverse=True)[:25]
    
    user_recommendations[user_id] = [book for book, _ in top_preds]
    
    if i % 500 == 0:
        print(f"Processed {i}/{len(all_user_ids)} users")

# ---------------------
# Save Recommendations to CSV
# ---------------------
with open('user_recommendations.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['user_id', 'recommended_books'])
    for user_id, books in user_recommendations.items():
        writer.writerow([user_id, ','.join(map(str, books))])

# ---------------------
# Save RMSE to a text file
# ---------------------
with open('svd_rmse.txt', 'w') as f:
    f.write(f"RMSE: {rmse:.4f}\n")

# ---------------------
# Optional: Load and display a preview
# ---------------------
df_recommendations = pd.read_csv('user_recommendations.csv')
print("\nSample recommendations:")
print(df_recommendations.head())


KeyboardInterrupt: 